In [ ]:
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np

from pykern.pkcollections import PKDict
import rslaser.rsoptics.wavefront as rswf
import rslaser.rscavity.laser_cavity as rslc

from srwlib import srwl

In [ ]:
k=PKDict()

#Define parameters for laser pulse starting at center of crystal
k.wavefrontEnergy = 1.55 #Wavefront Energy [eV]. 1.55 eV is 800 nm wavelength
k.lam = 1239.8*1e-9/k.wavefrontEnergy # convert energy [eV] to wavelength [m]
print('lam = %1.9f m' % k.lam)

k.sigrW = 0.000186   # rms size of laser pulse at waist

k.nslice = 5
k.sig_s = 0.1        # rms length of Gaussian laser pulse [m]

k.phE = k.wavefrontEnergy
k.energyChirp = .01

#create laser pulse
LP = rslc.LaserPulse(k)

In [ ]:
# choose one of the laser pulse slices, and grab its SRW wavefront object
slice=LP._slice
wfr0=slice[0]._wfr

In [ ]:
#Plot initial wavefront
##Initial wfr calculation
arIinit = srwlib.array('f', [0]*wfr0.mesh.nx*wfr0.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(arIinit, wfr0, 6, 0, 3, wfr0.mesh.eStart, 0, 0) #extracts intensity

##Reshaping electric field data from flat to 2D array
arIinit2D = np.array(arIinit).reshape((wfr0.mesh.nx, wfr0.mesh.ny), order='C')
wfrsizei=np.size(arIinit)

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvalsf02=np.linspace(wfr0.mesh.xStart,wfr0.mesh.xFin,wfr0.mesh.nx)
yvalsf02=np.linspace(wfr0.mesh.yStart,wfr0.mesh.yFin,wfr0.mesh.ny)

fig = plt.figure(figsize=(12,7))
ax = fig.gca()

# By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
plt.pcolormesh(np.multiply(xvalsf02,1e6), np.multiply(yvalsf02,1e6), arIinit2D, cmap=plt.cm.viridis,shading='auto')
plt.colorbar()
#['Horizontal Position [mm]', , 'Intensity']
ax.set_ylim(-1e4,1e4)
ax.set_xlim(-1e4,1e4)
ax.set_ylabel(r'Vertical Position [$\mu m$]')
ax.set_xlabel(r'Horizontal Position [$\mu m$]')
ax.set_title('Intensity at Waist')
plt.show()

In [ ]:
# create a Gaussian wavefront via SRW
propLen = 0.
pulseE = 1.
poltype = 0
wfr0=rswf.createGsnSrcSRW(k.sigrW,propLen,pulseE,poltype,k.phE)